#### 3.3.1 生成数据集

In [1]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

#### 3.3.2 读取数据

In [3]:
import torch.utils.data as Data
batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)

In [4]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[-0.3257,  0.1515],
        [ 0.2882, -0.4172],
        [-0.6543,  0.8479],
        [-0.9024, -0.9400],
        [ 0.8365, -1.1205],
        [-0.3871,  0.5195],
        [-2.1518,  0.4085],
        [ 0.5858, -1.6650],
        [ 0.6252,  2.2159],
        [ 1.4223,  1.1445]]) tensor([ 3.0279e+00,  6.2126e+00,  1.0936e-02,  5.5986e+00,  9.6811e+00,
         1.6637e+00, -1.4949e+00,  1.1032e+01, -2.1028e+00,  3.1582e+00])


#### 3.3.3 定义模型

In [7]:
import torch.nn as nn
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [8]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [9]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.6863,  0.3277]], requires_grad=True)
Parameter containing:
tensor([0.6579], requires_grad=True)


#### 3.3.4 初始化模型参数

In [10]:
from torch.nn import init
init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [11]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0042,  0.0140]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


#### 3.3.5 定义损失函数

In [12]:
loss = nn.MSELoss()

#### 3.3.6 定义优化算法

In [13]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [ ]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

In [ ]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

#### 3.3.7 训练模型

In [14]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000478
epoch 2, loss: 0.000136
epoch 3, loss: 0.000073


In [15]:
# 比较学到的模型参数和真实的模型参数。
# 从net获得需要的层，并访问其权重（weight）和偏差（bias）
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 2.0001, -3.4003]], requires_grad=True)
4.2 Parameter containing:
tensor([4.2000], requires_grad=True)
